In [0]:
# Uncomment and run the command below if you do not have pycrypto installed
# !pip install pycrypto

# Team A
### Nuhil Mehdy, Cody Valle, Patrick Chapman

---



## Description

---



This program is a secure multiparty auction based on the Yao's Millionaires' problem. It uses the idea of Mix and Match to implement a greater-than function for comparing bid pairs in an auction.

In [0]:
# imports
from Crypto import Random
from Crypto.Random import random
from Crypto.PublicKey import ElGamal
from Crypto.Util.number import GCD
from Crypto.Hash import SHA
from sympy import mod_inverse
from math import ceil, sqrt
import pandas as pd

## Initial Setup

---


 * key_size: The number of bits to be used for El Gamal
 * player_count: The number of bidders
 

---



Our auction is using two servers to perform the auction computations. First, we have to create a private key for each servers. Second, we generate a shared public key based on each private key.

In [0]:
key_size = 512
player_count = 10
bid_length = 8

# Building server 1
server_1_keys = ElGamal.generate(key_size, Random.new().read)
p = server_1_keys.p
g = server_1_keys.g
server_1_keys.p, server_1_keys.g, server_1_keys.x, server_1_keys.y

# Building server 2
server_2_x = random.randint(2, p-2)
server_2_y = pow(g, server_2_x, p)
server_2_keys = ElGamal.construct((p, g, server_2_y, server_2_x))
server_2_keys.p, server_2_keys.g, server_2_keys.x, server_2_keys.y

# Building group public key
group_pub_key = pow(server_1_keys.y * server_2_keys.y, 1, p)

# this is just so we can call encrypt easily
group_keys = ElGamal.construct((p, g, group_pub_key))

In [0]:
# A simple utility function for multiplying elements in a list
def multiplyList(myList): 
      
    # Multiply elements one by one 
    result = 1
    for x in myList: 
         result = pow((result * x), 1, p)
    return result 

## Baby-Step Giant-Step

---



We implemented a BSGS algorithm to solve the DLP problem. We have to use BSGS because we are using exponential El Gamal. BSGS is a computationally expensive task considering it must build dictionaries based on the size $\begin{equation} \sqrt(p)\end{equation}$

In [0]:
def bsgs(h, g, p):
  m = ceil(sqrt(p-1))
  
  #Big steps
  big = dict({pow(g, i, p) : i for i in range(m)})
  
  gm = pow(mod_inverse(g, p), m, p)
  
  #Little steps
  little = dict({pow(h * pow(gm, i, p), 1, p) : i for i in range(m)})
  
  for key in little:
    if key in big:
      return little[key] * m + big[key]
  return None

## Bidder Input

---



For the Mix and Match protocol, our input must be encrypted bitwise. Below are utility functions that generate a binary array for a big and another function that encrypts each individual bid in the array.

In [0]:
def getBinArray(bid):
  fstring = '{0:0'+str(bid_length)+'b}'
  binArray = [int(x) for x in list(fstring.format(bid))]
  
  if len(binArray) > bid_length:
     raise Exception('Bid size must be within 1 -'+str((2**bid_length)))
  
  return binArray

In [0]:
def getEncryptedArray(binArray, key):
  return [key.encrypt(pow(key.g, x, key.p), random.randint(2, key.p-2)) for x in binArray]

# PET

---


<p align="center">
The Plaintext Equality Test determines if two ciphertexts' plaintexts are actually equivalent. This idea is essential for our greater-than Mix and Match implementation later on. The idea is as follows:  

\begin{equation} y = g^{\sum{x_i}} \end{equation}    

Where $g$ is some generator, $y$ is a **shared** public key, and $x_i$ represents a party's individual private key.    

\begin{equation} (\alpha, \beta) = (my^{K_E}, g^{K_E})\end{equation}   
The resulting $(\alpha, \beta)$ represents the ciphertext. Where $m$ represents a plaintext message and $K_E$ represents an ephemeral key. Next, if we want to check if two plaintexts are equivalent $(\alpha, \beta) \equiv (\alpha ', \beta ')$ then the resulting $(\alpha / \alpha ', \beta / \beta ')$ should represent the encryption of 1. We can represent the value as: 
  
\begin{equation}(\epsilon, \zeta) \equiv  (\alpha / \alpha ', \beta / \beta ') \end{equation}  
However, we only want want to be able to determine if two values are equivalent so we must introduce a blind. In our case, each server chooses some random value $z_i$, then they each compute:  
  
\begin{equation} (\epsilon^{z_i}, \zeta^{z_i}) \equiv (\epsilon_i, \zeta_i) \end{equation}  
Together they must then jointly decrypt the value:  
  
\begin{equation} (\gamma, \delta) \equiv (\prod{\epsilon_i}, \prod{\zeta_i})\end{equation}  
If the plaintext results in the value of 1, we know that the two values are equivalent. Otherwise,  it is some random value and no further relationship can be determined.
  </p>

In [0]:
def PET(val1, val2):
  epsilon = pow(val1[1] * mod_inverse(val2[1], p), 1, p)
  zeta = pow(val1[0] * mod_inverse(val2[0], p), 1, p)
  #print(epsilon, zeta)
  
  
  server_1_blind = random.randint(2, p-2)
  server_2_blind = random.randint(2, p-2)
  #print(server_1_blind, server_2_blind)
  
  server_1_epsilon = pow(epsilon, server_1_blind, p)
  server_1_zeta = pow(zeta, server_1_blind, p)
  #print(server_1_epsilon, server_1_zeta)
  
  server_2_epsilon = pow(epsilon, server_2_blind, p)
  server_2_zeta = pow(zeta, server_2_blind, p)
  #print(server_2_epsilon, server_2_zeta)
  
  gamma = pow(server_1_epsilon * server_2_epsilon, 1, p)
  delta = pow(server_1_zeta * server_2_zeta, 1, p)
  #print(gamma, delta)
  
  Y_1 = pow(delta, server_1_keys.x, p)
  Y_2 = pow(delta, server_2_keys.x, p)
  #print(Y_1, Y_2)
  
  
  Y_prod = pow(Y_1 * Y_2, 1, p)
  
  decrypt = pow(gamma * mod_inverse(Y_prod, p), 1, p)
  #print(decrypt)
  
  return decrypt == 1

## Exhaustive Score Result

---

In the next section we can base the fact that a difference in score between bidders is either between 1 to 31 or some other seemingly random number. This means that we can easily determine the highest bidder in a two bidder comparison.

This is substantially faster than the BSGS. This is because our difference in score space (that we care about) between bidders is from 1 to 31. This means that we can simply use PET to compare the encrypted score value and all possible (again, that we care about) scores from 1 to 31.

In [0]:
def exhaustiveScoreCheck(score, key):
  for i in range((2**bid_length)):
    if(PET(key.encrypt(pow(key.g, i, key.p), random.randint(2, key.p-2)), score)):
      return True
    
  return False

## Building Tables

---



We need to build tables for Mix and Match to work. The tables represent the possible combinations for a single bit with two output columns representing the score that each player would receive depending on their provided bits. In our auction implementation, we are using 5-bits. This means that we would construct 5 different tables where the scoring columns would have values that are dependent on what bit index they are representing.

| B_1 | B_2 | OUT_1 | OUT_2 |
|-----|-----|-------|-------|
| 0   | 0   | 0     | 0     |
| 0   | 1   | 0     | X     |
| 1   | 0   | X     | 0     |
| 1   | 1   | 0     | 0     |  

For instance if B_2 is a 1, while B_1 is a 0, then B_2 would be awarded points based on the position of the index of the bit. In our implementation, we are representing the score as $2^{i-1}$ where i is the position of the bit from the bid.


In [0]:
def get_table(col1, col2, nth_bit):

  mix_net = pd.DataFrame({ 
      'B_1' : col1, 
      'B_2': col2,})
  
  mix_net['Out_1'] = mix_net['B_1'] > mix_net['B_2']
  mix_net['Out_2'] = mix_net['B_1'] < mix_net['B_2']  
  
  mix_net['Out_1'] = mix_net.Out_1.apply(lambda x: 2**(nth_bit-1) if x else 0)
  mix_net['Out_2'] = mix_net.Out_2.apply(lambda x: 2**(nth_bit-1) if x else 0)  
  
  return mix_net


# Assume each bid is 5 bits numnber
k = bid_length
table = []
for i in range(k):
  table.append(get_table([0, 0, 1, 1], [0, 1, 0, 1], k-i))

In [117]:
for t in table:
  print(t, "\n")

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0    128
2    1    0    128      0
3    1    1      0      0 

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0     64
2    1    0     64      0
3    1    1      0      0 

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0     32
2    1    0     32      0
3    1    1      0      0 

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0     16
2    1    0     16      0
3    1    1      0      0 

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0      8
2    1    0      8      0
3    1    1      0      0 

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0      4
2    1    0      4      0
3    1    1      0      0 

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0      2
2    1    0      2      0
3    1    1      0      0 

   B_1  B_2  Out_1  Out_2
0    0    0      0      0
1    0    1      0      

## Hiding Table Values

---



Note: We would normally combine this with the shuffling phase so that we can not keep track of the table positions. However, for demonstration purposes we are separating each part.

After we build the table we want to encrypt all the values of the table so that servers participating in the computation cannot figure out what the bidders' plaintext values are when performing the matching on the tables.

In [0]:
for t in table:
    t['Out_1'] = t.Out_1.apply(lambda x: group_keys.encrypt(pow(g, x, p), random.randint(2, p-2)))
    t['Out_2'] = t.Out_2.apply(lambda x: group_keys.encrypt(pow(g, x, p), random.randint(2, p-2)))
    t['B_1'] = t.B_1.apply(lambda x: group_keys.encrypt(pow(g, x, p), random.randint(2, p-2)))
    t['B_2'] = t.B_2.apply(lambda x: group_keys.encrypt(pow(g, x, p), random.randint(2, p-2)))   



In [119]:
for t in table:
  print(t, "\n")

                                                 B_1  \
0  (355610406492478167694913584782226379475021670...   
1  (267868201768692178047172825319260301760867195...   
2  (737038937872568169319174691643876607421996922...   
3  (282250873029587477842874868158657403086170257...   

                                                 B_2  \
0  (127085209006128885676079163577716530527221397...   
1  (607678320760644801138791551247375644895831974...   
2  (853138301598938673437276300637697354064390433...   
3  (478531840487864162180730004713570236120304261...   

                                               Out_1  \
0  (127909841622312021142503240613659536936808930...   
1  (106609070152222928831744446967259526801187144...   
2  (734505095875342747330370622634372723943294912...   
3  (651435518495232505001248222092236708022721043...   

                                               Out_2  
0  (485842534451905651302124415990471330979596472...  
1  (7226370400081478982305601466239955792972633

## Shuffling Table Rows

---

We must also shuffle the table rows so that servers cannot use the position of the rows to determine what the bidders' values are during the matching phase.

In [0]:
# For shuffling the rows
for i in range(len(table)):  
  table[i] = table[i].sample(frac=1).reset_index().drop(['index'], axis=1)                 

In [121]:
for t in table:
  print(t, "\n")

                                                 B_1  \
0  (282250873029587477842874868158657403086170257...   
1  (737038937872568169319174691643876607421996922...   
2  (355610406492478167694913584782226379475021670...   
3  (267868201768692178047172825319260301760867195...   

                                                 B_2  \
0  (478531840487864162180730004713570236120304261...   
1  (853138301598938673437276300637697354064390433...   
2  (127085209006128885676079163577716530527221397...   
3  (607678320760644801138791551247375644895831974...   

                                               Out_1  \
0  (651435518495232505001248222092236708022721043...   
1  (734505095875342747330370622634372723943294912...   
2  (127909841622312021142503240613659536936808930...   
3  (106609070152222928831744446967259526801187144...   

                                               Out_2  
0  (816174737704680549233561955718169510064547542...  
1  (6431938912705834447664748649990082408870008

## Getting Player Bids

---

In this section each player submits their bit as some integer less than 32. The result is a list of each player's bitwise encryptions.

In [122]:
player_bids = []
for i in range(player_count):
  print("Bidder", i)
  player_bids.append(getEncryptedArray(getBinArray(int(input("Enter a bid: "))), group_keys))

Bidder 0
Enter a bid: 16
Bidder 1
Enter a bid: 25
Bidder 2
Enter a bid: 1
Bidder 3
Enter a bid: 56
Bidder 4
Enter a bid: 240
Bidder 5
Enter a bid: 253
Bidder 6
Enter a bid: 10
Bidder 7
Enter a bid: 254
Bidder 8
Enter a bid: 4
Bidder 9
Enter a bid: 7


# Determining the Winner

---

In this section we are actually determining the winner. First, we set the highest bid to the first bidder. Next, we start iterating through all bidders.

We compare bids two at a time. We take each players encrypted bits and use the table to determine each bidders' score. We use PET on the first bidder and the first column in the Mix and Match table. We use PET on the second bidder and the second column in the MIX and Match table. When we find a row where PET passes for both cases, then we get the encrypted output values from the table and pass those as the score for that bit. After we compare each bit, we then are able to add the scores while keeping them encrypted because of our use of exponential El Gamal. We then subtract the bidders' scores against eachother using exponential El Gamal again. We can then use destributed decryption to get the appropriate $g^{score2-score1} mod p$. Once we get to this point we are able to solve the DLP and get the score. If the score is in the range 1-31 we know that score2 value won, otherwise the score1 value was higher. We can use this to determine the new highest bidder. We repeat this process for every bidder.

In [123]:
highest_bid = player_bids[0]


for bid in player_bids:
  i = 0 # start at most significant bit
  out_1 = []
  out_2 = []
  for bid_bit in bid:
    t = table[i]
    found = False
    
    for j in range(0, bid_length):
      if not found and PET(bid_bit, t.B_1[j]) and PET(highest_bid[i], t.B_2[j]):
        found = True
        out_1.append(t.Out_1[j])
        out_2.append(t.Out_2[j])
    i += 1 # python doesn't support ++?
  o_1_alpha = multiplyList(list(out_1[x][1] for x in range(bid_length)))
  o_1_beta = multiplyList(list(out_1[x][0] for x in range(bid_length)))
  o_2_alpha = multiplyList(list(out_2[x][1] for x in range(bid_length)))
  o_2_beta = multiplyList(list(out_2[x][0] for x in range(bid_length)))
    
  score_alpha = pow(o_2_alpha * mod_inverse(o_1_alpha, p), 1, p)
  score_beta = pow(o_2_beta * mod_inverse(o_1_beta, p), 1, p)
    
  server1_score_beta = pow(score_beta, server_1_keys.x, p)
  server2_score_beta = pow(score_beta, server_2_keys.x, p)
    
  decrypt_key = pow(server1_score_beta * server2_score_beta, 1, p)
  decrypt = pow(score_alpha * mod_inverse(decrypt_key, p), 1, p)
    
  #score = bsgs(decrypt, g, p)
  #if (score > 31):
  #  highest_bid = bid
    
    
  if(not exhaustiveScoreCheck((score_beta, score_alpha), group_keys)):
    highest_bid = bid
    
print("Bidder", player_bids.index(highest_bid), "is the winner!")

Bidder 7 is the winner!


## Limitations

When we are getting the score to determine who is the highest bidder, we are actually revealing information about the difference in their bid size. In this case, we are leaking pretty important information. An extension to this project would be to find a scoring system that would better hide the bidders' bids.

We are using exponential El Gamal, which means that we must solve a DLP (Discrete Logarithm Problem) in the end. We are using a small range of potential bids $2^5$, which means that we exhaustively determine the final score by performing PET on each potential encrypted score. We can also use BSGS to solve the DLP. The problem with BSGS is that it severly limits the key size that you can use when using El Gamal. This is because BSGS is dependent on the size of the modulus p. If p is large, (1024 for instance), Then we are going to be generating a list of around $\sqrt(2^{1024})$.